In [5]:
from transformers import pipeline
import torch
from transformers import DetrForObjectDetection, DetrImageProcessor, pipeline
import torchvision.transforms as transforms
import cv2 as cv
from PIL import Image
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import gradio as gr 
import requests
import numpy as np
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from helper import start_conversation, creating_conversation_with_blenderbot, produce_reply_sound

In [6]:
model_path = "./models/object-detection-resnet-50"
processor =  DetrImageProcessor.from_pretrained(model_path)
model = DetrForObjectDetection.from_pretrained(model_path)

In [ ]:
cap = cv.VideoCapture(0)
while True:    
    ret, frame = cap.read()
    if not ret:
        break
    frame_tensor = torch.as_tensor(frame, dtype=torch.float32)
    image_frame = processor(images=frame_tensor, return_tensors="pt")
    outputs= model(**image_frame)
    
    target_sizes = torch.tensor([[frame_tensor.shape[1], frame_tensor.shape[2]]])

    detections = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]
    for score, label, box in zip(detections["scores"], detections["labels"], detections["boxes"]):
        box = [int(i) for i in box.tolist()]
        cv.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2) 
        label_text = f"{model.config.id2label[label.item()]}: {round(score.item(), 2)}"
        cv.putText(frame, label_text, (box[0], box[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    if label_text.startswith ("person"):
        start_conversation()
        break
    cv.imshow("frame", frame)
    if cv.waitKey(1) == ord('q'):
         break
cap.release()
cv.destroyAllWindows()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


#### creating a conversation with someone
Reading the input from the microphone
detect the spoken langauge 
convert the microphone sampling rate to 16kz
convert the spoken words to text

##### creating conversation context using blenderbot model
from the transcribed spoken words create a conversation chat with
create a context to follow up the text messages converted and spoken

['i love doing NLP do u know anything?',
 " I don't know much about it, but I do know that it's a non-profit organization.",
 "i love doing NLP do u know anything?\n I don't know much about it, but I do know that it's a non-profit organization.\nNope i meant natural language processing",
 ' National Language Academy of the United States is a nonprofit organization that aims to educate and develop native languages.',
 "i love doing NLP do u know anything?\n I don't know much about it, but I do know that it's a non-profit organization.\ni love doing NLP do u know anything?\n I don't know much about it, but I do know that it's a non-profit organization.\nNope i meant natural language processing\n National Language Academy of the United States is a nonprofit organization that aims to educate and develop native languages.\noh really i didnt know about that then? so what do know about financial crisis",
 ' Financial crisis is a serious issue that affects a lot of people globally.']